# The Idea

While studying NLP an idea took root in my mind and began to blossom, unfolding in slow layers, and then displayed itself clearly.

When I was 9 years old my Dad was obsessed with assimilating to the ideas of respectibility; pantomiming the idea of status, through the thoughts and ideas of others. The consequence of which was a ritual of surrender conducted after supper prayer before popular daytime television host, Oprah Winfrey. 

Anything that she endorsed was taken as a communinon of sorts,and somehow, In the 5th grade I visited the local library of my small hometown checking out a book my parent's personal messiah recommended on her book list. A fruit that I'm certain my conservative southern baptist parents served me unknowingly...  'The Bluest Eye' by Toni Morrison. 

Up until this point I had only read Harry Potter series,the *Red Wall* series by Brian Jacques, and some other childhood favorites (The Boxcar Children, Junie B. Jones, A Wrinkle in Time, Black Beauty...etc.) . The limits of what I could conceive were primarily  defined by castles, wizards, talking animals...(Home life was bleak and demanding and I was cloying for escape)

However, reading this book, at the summit of girlhood and adolescence contextualized, personal truths that I had never been given, and that nobody cared  to (or maybe could?) share with me. Language became liberation. I was able to understand that I held value not just for how well I courted obedience, but for who I was.









In [81]:
!pip install datasets
!pip install transformers
!pip install gTTS
!pip install playsound
!pip install pyaudio
!apt-get install -y portaudio19-dev
!sudo apt-get install -y xvfb x11-utils
!sudo apt-get install -y x11vnc
!sudo apt-get install -y fluxbox
!sudo apt-get install -y ffmpeg
!sudo apt-get install -y xterm
!pip install gTTS
!pip install playsound
!pip install pyaudio
!apt-get install -y portaudio19-dev



# Start virtual display
!Xvfb :1 -screen 0 1024x768x24 &> xvfb.log  &
!export DISPLAY=:1

# Start audio
!pulseaudio --start

# Test audio
!paplay /usr/share/sounds/alsa/Front_Center.wav

!pip install openai
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 241, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 526, in run
    env = get_environment(lib_locations)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/metadata/__init__.py", line 85, in get_environment
    return select_backend().Environment.from_paths(paths)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/metadata/pkg_resources.py", line 232, in from_paths
    return cls(pkg_resources.WorkingSet(paths))
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 567, in __i

# Defining a model function to fine-tune our custom model:

* Intially the function takes four parameters: model_name, which specifies the name of the pre-trained GPT-2 model to use as a starting point; corpus_path, which is the path to a text file containing the custom corpus to use for fine-tuning; output_dir, which is the directory where the fine-tuned model and tokenizer will be saved; and epochs and batch_size, which specify the number of training epochs and the batch size to use for fine-tuning. 

* We then load the pre-trained GPT-2 model and tokenizer specified by model_name using the from_pretrained method provided by the Hugging Face Transformers library.

* The function then loads the custom corpus from the corpus_path file and tokenizes it using the pre-trained tokenizer. This is done using the TextDataset class provided by the Hugging Face Datasets library (They think of EVERYTHING). The tokenized data is split into chunks of 128 tokens (or fewer, if the end of the sequence is reached), which is specified by the block_size parameter.

* The function prepares a data collator using the DataCollatorForLanguageModeling class provided by the Hugging Face Transformers library. This data collator is used to prepare batches of data for training the fine-tuned model.

* We then set up  our training arguments using the TrainingArguments class provided by the Hugging Face Transformers library. These arguments specify various training settings, such as the output directory for the fine-tuned model, the number of training epochs, the batch size, and the learning rate.

* Then we launch a Trainer object using the Trainer class provided by the Hugging Face Transformers library. This trainer object is responsible for training the fine-tuned model using the specified dataset and training arguments.

* Lastly we call the '.train()' method to fine-tune the model and save it to the directory






In [103]:
# importing the necessary libaries

import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import requests
from io import StringIO
import speech_recognition as sr
import pyttsx3
import datetime
import random
import pyjokes
import requests
import json
from gtts import gTTS
import os
import openai
import pytz 



In [77]:
# defining a function to fine tune a gpt2 model
def fine_tune_gpt2(model_name, corpus_path, output_dir, epochs=10, batch_size=4):
    # Load pre-trained model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(model_name) # Load a pre-trained tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_name) # Load a pre-trained model

    # Load dataset
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=corpus_path,
        block_size=128,
    )

    # Prepare data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

  # Set up training arguments
    training_args = TrainingArguments(
      output_dir=output_dir,
      overwrite_output_dir=True,
      num_train_epochs=epochs,
      per_device_train_batch_size=batch_size,
      save_steps=10_000,
      save_total_limit=4,
      logging_dir='./logs',
      learning_rate=2e-5, # a common range for the learning rate when fine-tuning generative transformers like GPT-2 is 
                          # between 1e-5 and 5e-5 so I start with 3e-5 and move smaller
)

    # Create Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    # Train the model
    trainer.train()

    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)



# Creating the Corpus

I extracted the text information from some of Toni Morrison's novels using the python library PyPDF2  and reading them to a text file. 

I DO NOT OWN THE RIGHTS TO ANY OF THESE.






In [107]:

corpus_url = "https://raw.githubusercontent.com/JVCarmich0959/CSC228/main/Toni_corpus_.txt"


response = requests.get(corpus_url)
corpus_text = response.text


with open("temp.txt", "w") as f:
    f.write(corpus_text)



# Fine tuning the Model 

This takes a while!

In [108]:
# Fine tune GPT-2 model on a collection of texts from Toni Morrison
model_name = "gpt2-large"  # You can also use "gpt2-small" or "distilgpt2" to reduce memory requirements.
corpus_path = "temp.txt"
output_dir = "toni_morrison_gpt2"
fine_tune_gpt2(model_name, corpus_path, output_dir)

Step,Training Loss
500,3.314600
1000,3.194300
1500,2.954600
2000,2.745900
2500,2.734300
3000,2.417200
3500,2.363500
4000,2.293900
4500,2.016600
5000,2.046200


Now that we have a refined model...it's time to generate some text...

# Below I implement a class  named "GPT2Generator":

* **__init__(self, model_path)**: This is the class constructor method that initializes an instance of the class. It takes a single argument model_path, which is the path to the directory containing the fine-tuned GPT-2 model and tokenizer.

* **load_model(self, model_path)**: This method loads the fine-tuned GPT-2 model and tokenizer from the specified directory model_path. It returns a tuple containing the loaded model and tokenizer.

* **set_model_to_eval(self, model)**: This method sets the mode of the loaded model to evaluation mode.

* **generate_text(self, seed_text, num_words_to_generate=50, temperature=0.7)**: This is the main method of the class that generates text based on a given seed text. It takes three arguments: seed_text, which is the seed text for text generation, num_words_to_generate, which is the number of words to generate after the seed text, and temperature, which controls the degree of randomness in text generation. The method first tokenizes the seed text using the tokenizer, creates an attention mask, and generates text using the fine-tuned GPT-2 model. It then decodes the generated text and returns it.

In [109]:


class GPT2Generator:
    def __init__(self, model_path):
        self.model, self.tokenizer = self.load_model(model_path)
        self.set_model_to_eval(self.model)
    
    def load_model(self, model_path):
        model = GPT2LMHeadModel.from_pretrained(model_path)
        tokenizer = GPT2Tokenizer.from_pretrained(model_path)
        return model, tokenizer
    
    def set_model_to_eval(self, model):
        model.eval()
    
    def generate_text(self, seed_text, num_words_to_generate=50, temperature=0.7):
        # Tokenize the seed text
        input_ids = self.tokenizer.encode(seed_text, return_tensors="pt")

        # Create the attention mask
        attention_mask = torch.ones_like(input_ids)

        # Generate text
        output = self.model.generate(
            input_ids, 
            max_length=len(input_ids[0]) + num_words_to_generate, 
            num_return_sequences=1, # Change this to change the number of sentences to generate
            no_repeat_ngram_size=2,  # To prevent repetition of bi-grams
            do_sample=True, # To generate text randomly
            temperature=temperature, # To change the degree of randomness
            pad_token_id=self.tokenizer.eos_token_id,  # Set the pad_token_id
            attention_mask=attention_mask,        # Pass the attention mask
        )

        # Decode the generated text
        generated_text = self.tokenizer.decode(output[0], skip_special_tokens=True)

        return generated_text


In [116]:
# Create a GPT2Generator object
model_path = "toni_morrison_gpt2"
gpt2_generator = GPT2Generator(model_path)

# Generate text
seed_text = "She wailed at the top of her lungs "  
generated_text = gpt2_generator.generate_text(seed_text, num_words_to_generate=50, temperature=0.8)
print(generated_text)

She wailed at the top of her lungs  for what seemed like hours, her face a mask of pain she
never imagined she would wear. Finally she gave up and went into the yard to lie on the
grass. Her eyelids fell, but she thought she saw in the dark a


In [119]:
# Create a GPT-2Generator object
model_path = "toni_morrison_gpt2"
gpt2_generator = GPT2Generator(model_path)

# Prompt the model with a statement or question
prompt = "What do you think about the impact of literature on personal growth?"

# Generate a response in the style of Toni Morrison
generated_text = gpt2_generator.generate_text(prompt, num_words_to_generate=50, temperature=0.8)

# Print the response
print(generated_text)


What do you think about the impact of literature on personal growth?”
The class was very diverse. A few had taken the
college courses, most had not; most were old enough to have
died twenty years ago; a few were just starting out. But the distribution
of opinion was not


In [125]:
# Create a GPT-2Generator object
model_path = "toni_morrison_gpt2"
gpt2_generator = GPT2Generator(model_path)

# Prompt the model with a statement or question
prompt = input("Enter your text: ")

# Generate a response in the style of Toni Morrison
generated_text = gpt2_generator.generate_text(prompt, num_words_to_generate=50, temperature=0.8)

# Print the response
print(generated_text)

Enter your text: Underneath the shade of a tree
Underneath the shade of a tree, he said, “Is there a bathroom near here you don’t use?”
“No.‘But I ll go if you want me to. It�s near my work. Can you tell


With its ability to generate human-like text on a wide range of topics, the custom fine-tuned GPT-2 model could be used to generate content for novels, screenplays, and other forms of creative writing. ( I didn't mean for this to be THAT relevant but...) This has the potential to revolutionize the creative writing process by providing writers with a powerful new tool that can help them generate ideas and overcome writer's block.

The GPT-2 model can also be used for a wide range of NLP tasks, such as language translation, sentiment analysis, and even chatbot development. This makes it a powerful tool for automating tasks and generating responses in a human-like manner.

One potential use case for the custom fine-tuned GPT-2 model could be in  the field of digital memorialization. By using existing text data such as emails, social media posts, and other written content, we can generate new content that captures the unique voice and personality of a deceased person. This is a powerful way to keep their memory alive and to create a lasting legacy.

Another potential application of the custom fine-tuned GPT-2 model is customer service. With its ability to convey familiarity, the model could be used to generate responses to customer inquiries and complaints, freeing up human resources to focus on more complex issues.

The model could also be used in the field of education to generate study materials and practice quizzes. This has the potential to revolutionize the way students learn and study, providing them with personalized and engaging content that is tailored to their individual needs.

# More possibilities





In [126]:
tasks = {
    "time": "The current time is {time}",
    "joke": "Here's a joke for you: " + pyjokes.get_joke(),
    "weather": "The weather is currently {weather}",
    "news": "Here are the latest headlines: {news}"
}

def save_and_play_response(response_text, filename):
    tts = gTTS(text=response_text, lang='en')
    tts.save(filename)
    os.system(f"mpg321 {filename}")

# Define a function to perform predefined tasks based on input text
def perform_task(input_text):
    if "goodbye" in input_text.lower() or "bye" in input_text.lower():
        return None

    for task_name, task_response in tasks.items():
        if task_name in input_text:
            if task_name == "time":
                # Get the timezone for Eastern US
                eastern_tz = pytz.timezone('US/Eastern')
                current_time = datetime.datetime.now(eastern_tz).strftime("%I:%M %p")
                task_response = task_response.format(time=current_time)
            elif task_name == "weather":
                response = requests.get("https://api.openweathermap.org/data/2.5/weather?q=Goldsboro,us&appid=3d627a0bded23d61f00d343a21807bf8")
                weather_data = json.loads(response.text)
                weather_description = weather_data["weather"][0]["description"]
                task_response = task_response.format(weather=weather_description)
            elif task_name == "news":
                response = requests.get("https://newsapi.org/v2/top-headlines?country=us&apiKey=a5079ab552054b08bdb2b525c83adac6")
                news_data = json.loads(response.text)
                news_headlines = [article["title"] for article in news_data["articles"]]
                task_response = task_response.format(news="\n".join(news_headlines))
            elif task_name == "joke":
                task_response = "Here's a joke for you: " + pyjokes.get_joke()

            # Save and play the response
            save_and_play_response(task_response, "task_response.mp3")
            return "task_response.mp3"

    # If the input text doesn't match any predefined task, use GPT-3 to generate a response
    else:
        response_text = generate_response_gpt3(input_text)
        save_and_play_response(response_text, "gpt3_response.mp3")
        return "gpt3_response.mp3"


def listen():
    r = sr.Recognizer()
    wake_word = "toni"

    while True:
        input_text = input("Enter a command: ")

        if wake_word in input_text.lower():
            print("Hey, I'm TONI! How can I help you?")
            break

    input_text = input("Enter a command: ")
    response_text = perform_task(input_text)
    
    while response_text is not None:
        print(response_text)
        input_text = input("Enter a command: ")
        response_text = perform_task(input_text)

    save_and_play_response("Goodbye! Have a great day!", "goodbye_response.mp3")

if __name__ == "__main__":
    listen()


Enter a command: Hi Toni
Hey, I'm TONI! How can I help you?
Enter a command: joke
task_response.mp3
Enter a command: Who is Toni Morrison?
gpt3_response.mp3
Enter a command: Where is Wayne Community College?
gpt3_response.mp3
Enter a command: bye
